# Joins

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///
%sql PRAGMA foreign_keys = ON

# Die verschiedenen Joins

Es gibt verschiedene Arten Joins zu erklären. Ich bevorzuge immer noch folgendes Bild:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=SQL_Joins.png)



In [ ]:
%%sql

DROP TABLE IF EXISTS eins;
DROP TABLE IF EXISTS zwei;

CREATE TABLE eins(id INT PRIMARY KEY, val CHAR(1));
INSERT INTO eins VALUES(1,'A'), (2,'B'), (3,'C');
CREATE TABLE zwei(id INT PRIMARY KEY, val2 CHAR(1));
INSERT INTO zwei VALUES(2,'D'), (3,'E'), (4,'F');

# Aufgabe 1
Führen sie an den Tabellen `eins, zwei` verschiedene `JOINS` durch, um ein Gefühl für die veränderte Ergebnissmenge zu erhalten. Nachdem beide Tabellen gleich benannte `id` Attribute haben können Sie bei einem `JOIN` auf diese einen `NATURAL JOIN` oder `JOIN using(id)` oder `JOIN ON(eins.id = zwei.id)` verwenden.


#### Aufgabe 1.1 Implementieren Sie einen `INNER` Join in den drei oben genannten Varianten

#### Aufgabe 1.2 Implementieren Sie einen `LEFT Join`

#### Aufgabe 1.3 Implementieren Sie einen `RIGHT Join`

#### Aufgabe 1.4 Implementieren Sie `A - B`

#### Aufgabe 1.5 Implementieren Sie `B - A`

#### Aufgabe 1.5 Implementieren Sie einen `FULL OUTER JOIN`

# Aufgabe 2

Die Beispiele aus Aufgabe 1 waren bewusst abstrakt gewählt. Was heißt die jeweilige Form von Joins denn auf einen konkreten Use-Case? Was würde jede einzelne Join form bedeuten, wenn es um Stundenten und Bücher ginge, die Studenten ausleihen können?

- Inner Join: 
- Left Outer Join:
- Right Outer Join:
- Student - Buch:
- Buch - Student:
- FULL OUTER JOIN:

# Aufgabe 3

Betrachten wir folgene Modellierung:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bib.png)

In [ ]:
%%sql

DROP TABLE IF EXISTS leiht_aus;
DROP TABLE IF EXISTS ist_ueberfaellig;
DROP TABLE IF EXISTS student;
DROP TABLE IF EXISTS buch;

CREATE TABLE student(
    matrikelnummer INT PRIMARY KEY NOT NULL,
    name TEXT NOT NULL
);

CREATE TABLE buch(
    isbn TEXT PRIMARY KEY NOT NULL,
    name TEXT NOT NULL
);

CREATE TABLE leiht_aus(
    matrikelnummer INT NOT NULL,
    isbn TEXT NOT NULL,
    von DATE NOT NULL,
    bis DATE,
    PRIMARY KEY(matrikelnummer, isbn, von),
    FOREIGN KEY(matrikelnummer) REFERENCES student(matrikelnummer),
    FOREIGN KEY(isbn) REFERENCES buch(isbn)
);

CREATE TABLE ist_ueberfaellig(
    matrikelnummer INT NOT NULL,
    isbn TEXT NOT NULL,
    gebuer INT,
    PRIMARY KEY(matrikelnummer, isbn),
    FOREIGN KEY(matrikelnummer) REFERENCES student(matrikelnummer),
    FOREIGN KEY(isbn) REFERENCES buch(isbn)
);

In [ ]:
%%sql
INSERT INTO student VALUES(1,'Elser'), (2, 'Maier'), (3, 'Huber');
INSERT INTO buch VALUES(1,'Datenbanken'), (2, 'Ulysses'), (3, 'Das Niebelungenlied');
INSERT INTO ist_ueberfaellig VALUES(1,2,10), (2,2,15), (2,3,22);
INSERT INTO leiht_aus VALUES(1,1,'01.10.2021',NULL), (1,2,'01.11.2021',NULL), (2,3,'01.11.2021','11.11.2021');

#### Aufgabe 3.1

Entwickeln Sie einen JOIN, der angibt welche Studenten welche Bücher ausgeliehen haben und welche Gebüren da noch zu zahlen sind.

#### Aufgabe 3.2

Geben Sie nur ausleihen aus, die > 20 Euro Gebüren haben